In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from pathlib import Path

data_color = '/kaggle/input/plantvillage-dataset/color'
data_grayscale='/kaggle/input/plantvillage-dataset/grayscale'
data_segmented="/kaggle/input/plantvillage-dataset/segmented"

paths = [path.parts[-2:] for path in
         Path(data_color).rglob('*.*')]                             
df = pd.DataFrame(data=paths, columns=['Class','Images'])     
df = df.sort_values('Class',ascending=True)                   
df.reset_index(drop=True, inplace=True)                       

In [ ]:
class_names=df["Class"].unique()

species=[]
for i in class_names:
    name=i.split("_")
    if name[0] not in species:
        species.append(name[0])

print(f"Number of distinct species: {len(species)}")
print(species,'\n')

print(f"Number of distinct classes: {len(class_names)}")
print(class_names)

df["Class"].value_counts()

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(df['Class'],palette='muted')
plt.title('Distribution of Classes')
plt.show()

In [ ]:
def aspect_ratio(img):
    return img.shape[1]/img.shape[0]

def resolution(img):
    return f'{img.shape[1]}x{img.shape[0]}'  